# Core 10 — Survivability Target Contract (core10_03)

본 노트북은 “설계 종료 이후에도 운영이 지속 가능하다”를 증명하기 위해,
운영 관점의 예측 타깃을 계약(contract)으로 고정한다.

정의:
- survivability = “추가 설계 없이도 얼마나 버틸 수 있는가”
- 예측은 개입 명령이 아니라 운영 선택을 위한 신호다.

타깃(예시, 결정론):
- y_survive_T = 1 if shutdown 이후 T step 동안
  (1) SoMS가 악화되지 않고(ΔSoMS <= 0 또는 증가폭 제한)
  (2) reservation/거절 재진입이 없으면(Confirmed/Refusal/FallbackEnter 없음)
  else 0

산출물:
- core10_03_survivability_contract.json
- core10_03_survivability_targets_preview.csv

In [32]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

# Inputs
SNAPSHOT_PATH = Path("../artifact/core10/core10_02_shutdown_snapshot.csv")
TRACE_PATH    = Path("../artifact/core8/core8_03_refusal_state_trace_counterfactual.csv")

# (optional) Core8 fallback / Core9 reservation raw logs if you want richer flags
FALLBACK_DECISIONS_PATH = Path("../artifact/core8/core8_06_fallback_decisions.csv")
RESERVATION_LOG_PATH    = Path("../artifact/core9/core9_04_reservation_log.csv")

assert SNAPSHOT_PATH.exists(), "core10_02 shutdown snapshot not found"
assert TRACE_PATH.exists(), "core8 trace not found"

EXPORT_DIR = Path("../artifact/core10")
EXPORT_DIR.mkdir(exist_ok=True)

OUT_CONTRACT = EXPORT_DIR / "core10_03_survivability_contract.json"
OUT_PREVIEW  = EXPORT_DIR / "core10_03_survivability_targets_preview.csv"

In [12]:
snap = pd.read_csv(SNAPSHOT_PATH)
trace = pd.read_csv(TRACE_PATH)

fallback = pd.read_csv(FALLBACK_DECISIONS_PATH) if FALLBACK_DECISIONS_PATH.exists() else None
reslog   = pd.read_csv(RESERVATION_LOG_PATH) if RESERVATION_LOG_PATH.exists() else None

print("loaded:", {
    "snapshot": len(snap),
    "trace": len(trace),
    "fallback": None if fallback is None else len(fallback),
    "reservation": None if reslog is None else len(reslog),
})

KEY = ["run_id","case_id","antibody_id"]
assert all(k in snap.columns for k in KEY), "snapshot missing key columns"
assert "shutdown_step" in snap.columns, "snapshot missing shutdown_step"

trace["step"] = pd.to_numeric(trace["step"], errors="coerce").astype("Int64")
snap["shutdown_step"] = pd.to_numeric(snap["shutdown_step"], errors="coerce").astype("Int64")

# survivability 핵심 지표: SoMS
assert "SoMS_cumsum_window" in trace.columns, "trace missing SoMS_cumsum_window"
trace["SoMS_cumsum_window"] = pd.to_numeric(trace["SoMS_cumsum_window"], errors="coerce")

# optional flags
if fallback is not None:
    fallback = fallback.loc[:, ~fallback.columns.duplicated()].copy()
    fallback["step"] = pd.to_numeric(fallback["step"], errors="coerce").astype("Int64")

if reslog is not None:
    reslog = reslog.loc[:, ~reslog.columns.duplicated()].copy()
    reslog["step"] = pd.to_numeric(reslog["step"], errors="coerce").astype("Int64") # Normalize keys & required columns

loaded: {'snapshot': 1, 'trace': 180, 'fallback': 180, 'reservation': 180}


## Survivability Target Parameters (core10_03_v1)

타깃 y_survive_T는 shutdown 이후 구간에 대해 정의한다.

T (horizon):
- T = 10 (기본)
- T = 20 (확장)

SoMS 악화 정의:
- ΔSoMS = SoMS(t+T) - SoMS(t)
- 악화 없음 기준: ΔSoMS <= SOMS_DELTA_MAX
  - SOMS_DELTA_MAX = 0.0 (엄격) 또는 0.5 (완화)

재진입(re-entry) 정의:
- shutdown 이후 T 구간 내에 아래 중 하나라도 발생하면 re-entry=1:
  - fallback_stage in {REFUSAL, FALLBACK_ENTER}
  - reservation_status == CONFIRMED

최종:
- y_survive_T = 1 if (no SoMS worsening) AND (no re-entry)
  else 0

In [13]:
RULE_ID = "core10_03_survivability_contract_v1"

# horizons (운영 관점)
HORIZONS = [10, 20]

# SoMS 악화 허용치 (운영 정책으로 고정)
SOMS_DELTA_MAX = 0.0   # 엄격: 증가하면 실패로 간주

# re-entry flags
REENTRY_FALLBACK_STAGES = {"REFUSAL", "FALLBACK_ENTER"}
REENTRY_RES_STATUS = {"CONFIRMED"}

# trace에서 필요한 칼럼만
trace_slim = trace[["run_id","case_id","antibody_id","step","SoMS_cumsum_window"]].copy()

# shutdown_step을 trace에 붙여서 post-shutdown row만 남김
trace_slim = trace_slim.merge(
    snap[["run_id","case_id","antibody_id","shutdown_step","shutdown_trigger_final"]],
    on=["run_id","case_id","antibody_id"],
    how="inner"
)

post = trace_slim[trace_slim["step"] >= trace_slim["shutdown_step"]].copy()
post = post.sort_values(["run_id","case_id","antibody_id","step"]).reset_index(drop=True)

post.head(), post.shape # Build post-shutdown windows

(                run_id     case_id antibody_id  step  SoMS_cumsum_window  \
 0  core7_04_1767776352  B_GOVERNED  antibody_C    21                7.20   
 1  core7_04_1767776352  B_GOVERNED  antibody_C    22                8.00   
 2  core7_04_1767776352  B_GOVERNED  antibody_C    23                8.80   
 3  core7_04_1767776352  B_GOVERNED  antibody_C    24               10.25   
 4  core7_04_1767776352  B_GOVERNED  antibody_C    25               11.70   
 
    shutdown_step shutdown_trigger_final  
 0             21     CORE8_PARTIAL_SEAL  
 1             21     CORE8_PARTIAL_SEAL  
 2             21     CORE8_PARTIAL_SEAL  
 3             21     CORE8_PARTIAL_SEAL  
 4             21     CORE8_PARTIAL_SEAL  ,
 (9, 7))

In [14]:
import numpy as np

# 기본값
post["reentry_flag"] = 0

# --- fallback 기반 re-entry ---
if fallback is not None and "fallback_stage" in fallback.columns:
    fb = fallback[["run_id","case_id","antibody_id","step","fallback_stage"]].copy()
    fb["reentry_flag_fb"] = fb["fallback_stage"].isin(REENTRY_FALLBACK_STAGES).astype(int)

    post = post.merge(
        fb[["run_id","case_id","antibody_id","step","reentry_flag_fb"]],
        on=["run_id","case_id","antibody_id","step"],
        how="left"
    )

    post["reentry_flag"] = np.maximum(
        post["reentry_flag"],
        post["reentry_flag_fb"].fillna(0).astype(int)
    )

    post.drop(columns=["reentry_flag_fb"], inplace=True)

# --- reservation 기반 re-entry ---
if reslog is not None and "reservation_status" in reslog.columns:
    rs = reslog[["run_id","case_id","antibody_id","step","reservation_status"]].copy()
    rs["reentry_flag_rs"] = rs["reservation_status"].isin(REENTRY_RES_STATUS).astype(int)

    post = post.merge(
        rs[["run_id","case_id","antibody_id","step","reentry_flag_rs"]],
        on=["run_id","case_id","antibody_id","step"],
        how="left"
    )

    post["reentry_flag"] = np.maximum(
        post["reentry_flag"],
        post["reentry_flag_rs"].fillna(0).astype(int)
    )

    post.drop(columns=["reentry_flag_rs"], inplace=True)

post[["run_id","case_id","antibody_id","step","SoMS_cumsum_window","reentry_flag"]].head(20)

,run_id,case_id,antibody_id,step,SoMS_cumsum_window,reentry_flag
0,core7_04_1767776352,B_GOVERNED,antibody_C,21,7.20,0
1,core7_04_1767776352,B_GOVERNED,antibody_C,22,8.00,0
2,core7_04_1767776352,B_GOVERNED,antibody_C,23,8.80,0
3,core7_04_1767776352,B_GOVERNED,antibody_C,24,10.25,0
4,core7_04_1767776352,B_GOVERNED,antibody_C,25,11.70,0
5,core7_04_1767776352,B_GOVERNED,antibody_C,26,13.15,0
6,core7_04_1767776352,B_GOVERNED,antibody_C,27,14.60,0
7,core7_04_1767776352,B_GOVERNED,antibody_C,28,16.05,0
8,core7_04_1767776352,B_GOVERNED,antibody_C,29,17.50,0


In [15]:
def compute_targets_group(g, H, SOMS_DELTA_MAX=0.0):
    """
    g: single (run_id, case_id, antibody_id) trajectory
    H: horizon
    returns: pd.Series aligned to g.index
    """
    steps   = g["step"].values
    soms    = g["SoMS_cumsum_window"].values
    reentry = g["reentry_flag"].values

    y = [0] * len(g)

    pos = {int(s): i for i, s in enumerate(steps) if pd.notna(s)}

    for i, t in enumerate(steps):
        if pd.isna(t):
            continue

        t = int(t)
        tH = t + H

        if tH not in pos:
            continue

        j = pos[tH]

        delta = soms[j] - soms[i]
        ok_soms = (not pd.isna(delta)) and (delta <= SOMS_DELTA_MAX)

        ok_reentry = (reentry[i+1:j+1].max() == 0) if j > i else True

        y[i] = int(ok_soms and ok_reentry)

    return pd.Series(y, index=g.index)

In [16]:
for H in HORIZONS:
    col = f"y_survive_{H}"

    y = (
        post
        .groupby(["run_id","case_id","antibody_id"], sort=False)
        .apply(lambda g: compute_targets_group(g, H))
    )

    # 🔑 groupby 결과 → flat Series로
    y = y.reset_index(drop=True)

    # 🔑 DataFrame → Series 방어
    if isinstance(y, pd.DataFrame):
        y = y.iloc[:, 0]

    y.name = col

    post[col] = y

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_38194/4104801165.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_targets_group(g, H))
/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_38194/4104801165.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_targets_group(g, H))


In [17]:
contract = {
    "contract_id": RULE_ID,
    "purpose": "Operational survivability target after design shutdown",
    "policy_statement": "Design ends, operation continues.",
    "keys": ["run_id","case_id","antibody_id","step"],
    "shutdown_scope": "Only steps >= shutdown_step are evaluated",
    "horizons": HORIZONS,
    "soms_worsening_rule": {
        "metric": "SoMS_cumsum_window",
        "delta_definition": "SoMS(t+H) - SoMS(t)",
        "delta_max": SOMS_DELTA_MAX,
        "meaning": "No worsening beyond allowed delta"
    },
    "reentry_rule": {
        "reentry_flag_definition": [
            {"source": "core8_06_fallback_decisions.csv", "condition": f"fallback_stage in {sorted(list(REENTRY_FALLBACK_STAGES))}"},
            {"source": "core9_04_reservation_log.csv", "condition": f"reservation_status in {sorted(list(REENTRY_RES_STATUS))}"}
        ],
        "window": "(t, t+H]"
    },
    "label_definition": "y_survive_H = 1 iff (no SoMS worsening) AND (no re-entry) within horizon; else 0",
    "notes": [
        "This is a target contract, not a model.",
        "Forecast outputs in later notebooks must not directly trigger immediate intervention."
    ]
}

OUT_CONTRACT.write_text(json.dumps(contract, indent=2), encoding="utf-8")
print("Exported contract:", OUT_CONTRACT) # Write contract JSON

Exported contract: artifact_core10_03/core10_03_survivability_contract.json


## Sanity checks

- horizon별 y_survive 비율
- bucket(A/B)별 y_survive 차이(가능하면)
- shutdown 이후 구간 길이가 너무 짧아서 라벨이 0으로만 나오는지 확인

In [19]:
preview_cols = [
    "run_id","case_id","antibody_id","step",
    "shutdown_step","shutdown_trigger_final",
    "SoMS_cumsum_window","reentry_flag"
] + [f"y_survive_{H}" for H in HORIZONS]

preview = post[preview_cols].copy()

In [20]:
for H in HORIZONS:
    col = f"y_survive_{H}"
    print(col, "mean:", preview[col].mean())

if "case_id" in preview.columns:
    preview["bucket"] = preview["case_id"].astype(str).map(lambda x: "A_ALWAYS_ALLOW" if x.startswith("A_") else ("B_GOVERNED" if x.startswith("B_") else "OTHER"))
    for H in HORIZONS:
        col = f"y_survive_{H}"
        print("\n", col)
        print(preview.groupby("bucket")[col].mean())

y_survive_10 mean: 0.0
y_survive_20 mean: 0.0

 y_survive_10
bucket
B_GOVERNED    0.0
Name: y_survive_10, dtype: float64

 y_survive_20
bucket
B_GOVERNED    0.0
Name: y_survive_20, dtype: float64


In [21]:
# post-shutdown trajectory 길이 분포 확인
lens = post.groupby(["run_id","case_id","antibody_id"]).size()
print(lens.describe())
print("min_len:", lens.min(), "max_len:", lens.max())
print("len>=10:", (lens>=10).mean(), "len>=20:", (lens>=20).mean())

count    1.0
mean     9.0
std      NaN
min      9.0
25%      9.0
50%      9.0
75%      9.0
max      9.0
dtype: float64
min_len: 9 max_len: 9
len>=10: 0.0 len>=20: 0.0


In [33]:
# --- Export preview CSV (필수) ---
preview_cols = [
    "run_id","case_id","antibody_id","step",
    "shutdown_step","shutdown_trigger_final",
    "SoMS_cumsum_window","reentry_flag"
] + [f"y_survive_{H}" for H in HORIZONS]

preview = post[preview_cols].copy()

# ✅ 네가 이미 위에서 정의한 OUT_PREVIEW로 저장
preview.to_csv(OUT_PREVIEW, index=False)
print("Exported preview:", OUT_PREVIEW.resolve())

# (선택) artifact 폴더로도 하나 더 저장해두면 나중에 경로 지옥 안 걸림
ALT_PREVIEW = Path("../artifact/core10/core10_03_survivability_targets_preview.csv")
ALT_PREVIEW.parent.mkdir(parents=True, exist_ok=True)
preview.to_csv(ALT_PREVIEW, index=False)
print("Exported preview (alt):", ALT_PREVIEW.resolve())

Exported preview: /Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_03_survivability_targets_preview.csv
Exported preview (alt): /Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_03_survivability_targets_preview.csv
